In [104]:
import sys
sys.path.append("..")  

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from util import map_bin_to_potentialness

PATH_TO_DATA = "../data/"
tag = "random_learning_stepsize"

In [105]:
n_agents, n_actions = 2, 2
n_bins = 20
learner = "mirror_ascent(entropic)"

setting = f"random_matrix_game_uniform_{n_agents}_{n_actions}"
df = pd.read_csv(f"{PATH_TO_DATA}{tag}/{learner}_{setting}.csv")
df["potentialness"] = map_bin_to_potentialness(df["bin"], n_bins)


In [106]:
# average over runs
df = df.groupby(["seed", "eta"]).agg({"convergence": "mean", "potentialness": "first"}).reset_index()
df

,seed,eta,convergence,potentialness
0,3028,128.00000,0.5,0.575
1,6351,0.03125,0.0,0.425
2,6351,0.12500,0.0,0.425
3,6351,0.50000,0.0,0.425
4,6351,1.00000,0.0,0.425
...,...,...,...,...
2063,990015,1.00000,0.0,0.175
2064,990015,2.00000,0.0,0.175
2065,990015,8.00000,0.0,0.175
2066,990015,32.00000,0.0,0.175


In [107]:
# Combine into a single groupby command
df = df.loc[df.groupby('seed')['convergence'].idxmax()]  # Get rows corresponding to max 'convergence' in each group


In [111]:
df[df.convergence==1].eta.unique()

array([128.])

In [99]:
df[df.eta == 0.03125]

,seed,eta,convergence,potentialness
1,6351,0.03125,0.0,0.425
10,8618,0.03125,0.0,0.275
20,18534,0.03125,0.0,0.425
30,26278,0.03125,0.0,0.075
38,28815,0.03125,0.0,0.025
...,...,...,...,...
2024,982074,0.03125,0.0,0.325
2032,982396,0.03125,0.0,0.375
2041,986698,0.03125,0.0,0.375
2052,989322,0.03125,0.0,0.375


In [ ]:

df['eta'] = df.groupby('seed')['eta'].transform('first')  # Get the 'eta' value corresponding to the first row in each group

In [100]:
df[df.seed == 26278]

,seed,eta,convergence,potentialness
30,26278,0.03125,0.0,0.075


In [84]:
for s in df.seed.unique():
    df.loc[df.seed == s,"eta"] = df["eta"][df[df.seed == s]["convergence"].idxmax()]
df = df.groupby("seed").first()

In [82]:
df

,seed,eta,convergence,potentialness
0,3028,128.00000,0.5,0.575
1,6351,0.03125,0.0,0.425
2,6351,0.03125,0.0,0.425
3,6351,0.03125,0.0,0.425
4,6351,0.03125,0.0,0.425
...,...,...,...,...
2063,990015,0.03125,0.0,0.175
2064,990015,0.03125,0.0,0.175
2065,990015,0.03125,0.0,0.175
2066,990015,0.03125,0.0,0.175


In [68]:
df.groupby('seed').idxmax()

,eta,convergence,potentialness
seed,,,
3028,0,0,0
6351,8,1,1
8550,9,9,9
8618,17,10,10
17249,18,18,18
...,...,...,...
987015,2049,2049,2049
988348,2050,2050,2050
988601,2051,2051,2051


In [69]:

# only keep the best learning rate for each seed
df = df.groupby("seed").agg({"convergence": "max", "potentialness": "first", "eta": lambda x: x.loc[df['convergence'].idxmax()]})
df

KeyError: 9

In [71]:
df.loc[9]

seed             8550.000
eta               128.000
convergence         1.000
potentialness       0.875
Name: 9, dtype: float64

In [74]:
df.head(20)

,seed,eta,convergence,potentialness
0,3028,128.00000,0.5,0.575
1,6351,0.03125,0.0,0.425
2,6351,0.12500,0.0,0.425
3,6351,0.50000,0.0,0.425
4,6351,1.00000,0.0,0.425
5,6351,2.00000,0.0,0.425
6,6351,8.00000,0.0,0.425
7,6351,32.00000,0.0,0.425
8,6351,128.00000,0.0,0.425
9,8550,128.00000,1.0,0.875


In [51]:
df[df.seed == 6351][["convergence"]].idxmax()

convergence    1
dtype: int64

In [46]:
df[df.seed == 6351]

,seed,eta,convergence,potentialness
1,6351,0.03125,0.0,0.425
2,6351,0.12500,0.0,0.425
3,6351,0.50000,0.0,0.425
4,6351,1.00000,0.0,0.425
5,6351,2.00000,0.0,0.425
6,6351,8.00000,0.0,0.425
7,6351,32.00000,0.0,0.425
8,6351,128.00000,0.0,0.425


In [25]:
# finally group by potentialness and average convergence over seeds
df = df.groupby(["potentialness"]).agg({"convergence": "mean"}).reset_index()

,potentialness,convergence
0,0.025,0.000000
1,0.075,0.000000
2,0.125,0.000000
3,0.175,0.000000
4,0.225,0.000000
5,0.275,0.000000
6,0.325,0.000000
7,0.375,0.070769
8,0.425,0.352727
9,0.475,0.708571


In [10]:
df = (
    df.groupby(["potentialness"]).agg({"convergence": "mean"}).reset_index()
)